In [1]:
import pandas
import configparser
import psycopg2

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['database']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

In [4]:

conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

In [5]:

table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp3c
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [6]:
cur.execute(table_sql)
conn.commit()

In [33]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=20'

In [34]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
print(html_data.decode('utf-8'))

<!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<meta http-equiv="content-type" content="text/html;charset=UTF-8">
<script type="text/javascript" src="/s/798993d/en_US.js"></script>
<script>window.indeed=window.indeed||{};indeed.crashtext=indeed.crashtext||{};indeed.crashtext.populate=indeed.crashtext.populate||function(c,g){var d=window[g];if(d){var e=d[""]||{},h=e.salt;if(e.hasOwnProperty("salt"))for(var b in c){for(var a=void 0,a=h,f=b.length;f;)a=33*a^b.charCodeAt(--f);a>>>=0;e.hasOwnProperty("id_length")&&(a=String(a).substring(0,e.id_length));d[a]=c[b]}else for(b in c)d[b]=[null].concat(c[b])}};indeed.crashtext.populate({"indeedapply_serp_label":["Apply instantly"],"See new jobs for this search":["Get email updates on this search"]}, 'indeed.i18n.localeData')</script>
<link href="/s/60d7c47/jobsearch_all.css" rel="stylesheet" type="text/css">
<link rel="alternate" type="application/rss+xml" title="Intelligence Analyst Jobs, Employment" href="http://rss.indeed.com/rss?q=intellig

In [35]:

!pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup

In [36]:

for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [37]:

for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [38]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

In [39]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp3c(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

In [40]:
df = pandas.read_sql_query('select * from indeed_gp3c', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Undergraduate Internship/Co-op Program - Intel...,Central Intelligence Agency,"Washington, DC",$21.21 - $22.43 an hour,"Engineering, science students, or those in oth..."
1,2,Intelligence Senior Analyst,Citi,"Tampa, FL 33646",None,If you are a person with a disability and need...
2,3,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Analyst can identify risks in security sys...
3,4,H-ISAC Cyber Threat Intelligence Analyst,Perch Security,"Titusville, FL","$50,000 - $75,000 a year",Ability to communicate technical issues to tec...
4,5,Brand Intelligence Analyst,LookingGlass Cyber Solutions,"Reston, VA 20191",None,The Brand Intelligence Analyst employs our pro...
5,6,Quantitative Analyst / Developer,RT Signal,"New York, NY 10013 (Tribeca area)","$95,000 - $165,000 a year","We are a small, but well-funded (financially s..."
6,7,Crisis Response Intelligence Analyst,Pinkerton,None,None,"The Crisis Response Intelligence Analyst, assi..."
7,8,Intelligence Analyst,Information Network Associates (INA),None,None,"Environments, with a willingness to develop sp..."
8,9,CIRT/Forensics/Threat Intelligence Analyst,Capgemini,None,None,All qualified applicants will receive consider...
9,10,"Police Intelligence Analyst, Part-Time",Allegheny County Police,None,"$1,907 a month",Please note that you MUST submit an applicatio...
